# Reflection and Blogpost Writing

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
#llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}
#llm_config = {"model": "gemini-1.5-flash","api_key":"PutYourKey","api_type":"google"}
llm_config = {"model": "llama3","api_type":"ollama"}


## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Intelligence.make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task ,"role": "user"}])

In [5]:
print(reply)

{'content': '**"Unlocking Human Potential: The Promise of Hope AI"**\n\nIn a world where technology and humanity converge, Hope AI is revolutionizing the way we live and work. This cutting-edge platform leverages artificial intelligence to amplify human capabilities, empowering individuals to achieve their full potential. With its proprietary algorithms and machine learning techniques, Hope AI helps users prioritize tasks, streamline workflows, and make data-driven decisions. By automating mundane tasks and freeing up mental bandwidth, Hope AI is poised to transform industries and redefine the future of work. Join the movement and discover a world where technology amplifies human ingenuity.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Human Potential: The Power of Hope AI**

In an era where technology and humanity coexist, Hope AI emerges as a beacon of innovation. This groundbreaking platform harnesses the collective wisdom of human emotions to fuel creative problem-solving. By leveraging emotional intelligence, Hope AI empowers individuals to tackle complex challenges with empathy and compassion. Imagine a world where AI-driven decision-making prioritizes human well-being, fostering a more harmonious society. With Hope AI, we're on the cusp of a revolution that combines the best of both worlds – technology and humanity. The future is looking brighter than ever.

--------------------------------------------------------------------------------
Critic (to Wr

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Unlocking Human Potential: The Power of Hope AI**

In an era where technology and humanity coexist, Hope AI emerges as a beacon of innovation. This groundbreaking platform harnesses the collective wisdom of human emotions to fuel creative problem-solving. By leveraging emotional intelligence, Hope AI empowers individuals to tackle complex challenges with empathy and compassion. Imagine a world where AI-driven decision-making prioritizes human well-being, fostering a more harmonious society. With Hope AI, we're on the cusp of a revolution that combines the best of both worlds – technology and humanity. The future is looking brighter than ever.

--------------------------------------------------------------------------------

************

## Get the summary

In [15]:
print(res.summary)

**Unlocking Human Potential: The Power of Hope AI**

Discover how Hope AI combines human emotions and technology to empower creative problem-solving and drive positive change in society.

Hope AI is revolutionizing the way we approach complex challenges by harnessing the collective wisdom of human emotions. By leveraging emotional intelligence, this groundbreaking platform empowers individuals to tackle problems with empathy and compassion. Imagine a world where AI-driven decision-making prioritizes human well-being, fostering a more harmonious society.

While Hope AI has numerous benefits, it's essential to acknowledge its limitations and potential drawbacks. Nevertheless, the platform's ability to drive positive change is undeniable. With credible evidence and scientific research backing up its effectiveness, Hope AI is poised to make a significant impact in various industries and aspects of our lives.

**Specific Use Cases:**

1. **Mental Health Support:** Hope AI can help individua